# Overview

This exercise produces one clean, validated dataset, transformed into .csv format with data organized into columns and rows. The final dataset will be ready for further analysis in Python, SQL, and Tableau.

The source data consists of six .fit (Flexible and Interoperable Data Transfer) files — a binary format developed by Garmin for storing fitness activity data. These files contain timestamped records such as heart rate, speed, cadence, GPS position, and other sensor metrics captured during a workout. Each file is structured as a sequence of messages, with "record" messages representing individual data points — typically one per second.
In this notebook, we will import the files and transform them step by step, documenting each stage with Markdown and inline comments.

fitparse is a third-party Python library used to extract structured data from .fit files. It’s not included in standard Python distributions, so it must be installed first.

In [5]:
# Install the fitparse library to enable parsing of .fit files
pip install fitparse

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Import common libraries and fitparse
import pandas as pd
import numpy as np
import datetime as dt
from fitparse import FitFile

The files below are saved in the same directory as this notebook and so there, no file paths are required. 

In [8]:
# Load the fitfiles
ar1 = FitFile('Aerobic_Recovery_1.fit')
ar2 = FitFile('Aerobic_Recovery_2.fit')
ar3 = FitFile('Aerobic_Recovery_3.fit')
ar4 = FitFile('Aerobic_Recovery_4.fit')
ar5 = FitFile('Aerobic_Recovery_5.fit')
ar6 = FitFile('Aerobic_Recovery_6.fit')

I’m interested in messages of type "record" — these are second-by-second snapshots of my ride, containing measurements like heart rate, cadence, speed, and power.
Each "record" is a structured object containing multiple fields. I loop through each one and extract its contents as a dictionary, where:

- data.name is the field label (e.g. "heart_rate")
- data.value is the actual measurement (e.g. 132)

This gives me a list full of dictionaries, one dictionary per timestamped data point.

In [11]:
# Extract 'record' messages from each file and convert to list of dictionaries

records1 = [{data.name: data.value for data in record} for record in ar1.get_messages("record")]
records2 = [{data.name: data.value for data in record} for record in ar2.get_messages("record")]
records3 = [{data.name: data.value for data in record} for record in ar3.get_messages("record")]
records4 = [{data.name: data.value for data in record} for record in ar4.get_messages("record")]
records5 = [{data.name: data.value for data in record} for record in ar5.get_messages("record")]
records6 = [{data.name: data.value for data in record} for record in ar6.get_messages("record")]

In [13]:
# Convert each list of dictionaries into a DataFrame
ardf1 = pd.DataFrame(records1)
ardf2 = pd.DataFrame(records2)
ardf3 = pd.DataFrame(records3)
ardf4 = pd.DataFrame(records4)
ardf5 = pd.DataFrame(records5)
ardf6 = pd.DataFrame(records6)

We will now stack the six individual DataFrames to create one master DataFrame containing all records. This will allow us to validate and clean the data in a single structure. The index will be reset to ensure a continuous row sequence across all workouts.

In [15]:
# Create one DataFrame.
ardf_master = pd.concat([ardf1, ardf2, ardf3, ardf4, ardf5, ardf6])

The files were imported in the order they were created. That said, this may not always be the case — so to ensure consistency, we sort the combined dataset by timestamp. This guarantees that all records are in true chronological order, regardless of file import sequence.

In [23]:
# Sort the master DataFrame by timestamp
ardf_master = ardf_master.sort_values(by="timestamp").reset_index(drop=True)

In [ ]:
Carry out validation of the new dataframe

In [25]:
# View the number of columns and rows
ardf_master.shape

(21606, 11)

In [32]:
# View column names, data types and non-null counts
ardf_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21606 entries, 0 to 21605
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   accumulated_power  21605 non-null  float64       
 1   activity_type      21606 non-null  object        
 2   cadence            21605 non-null  float64       
 3   distance           21606 non-null  float64       
 4   enhanced_speed     21606 non-null  float64       
 5   heart_rate         21606 non-null  int64         
 6   power              21605 non-null  float64       
 7   speed              21606 non-null  float64       
 8   timestamp          21606 non-null  datetime64[ns]
 9   position_lat       0 non-null      object        
 10  position_long      0 non-null      object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(3)
memory usage: 1.8+ MB


There are two columns named 'position_lat' and 'position_long' that are unecessary for this study, whichh is a stationary indoor trainer ride. We can delete these columns.

In [33]:
# Drop unused GPS columns
ardf_master = ardf_master.drop(columns=["position_lat", "position_long"])

In [34]:
# Check the summary of the modified dataframe
ardf_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21606 entries, 0 to 21605
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   accumulated_power  21605 non-null  float64       
 1   activity_type      21606 non-null  object        
 2   cadence            21605 non-null  float64       
 3   distance           21606 non-null  float64       
 4   enhanced_speed     21606 non-null  float64       
 5   heart_rate         21606 non-null  int64         
 6   power              21605 non-null  float64       
 7   speed              21606 non-null  float64       
 8   timestamp          21606 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 1.5+ MB


If we confirm there are no nulls to deal with, we can continue to build the final master datarrame

In [35]:
ardf_master.head()

,accumulated_power,activity_type,cadence,distance,enhanced_speed,heart_rate,power,speed,timestamp
0,139.0,cycling,74.0,3.58,3.583,90,139.0,3.583,2023-12-05 12:30:35
1,268.0,cycling,78.0,7.50,3.917,91,129.0,3.917,2023-12-05 12:30:36
2,377.0,cycling,80.0,11.72,4.222,93,109.0,4.222,2023-12-05 12:30:37
3,468.0,cycling,84.0,16.25,4.528,93,91.0,4.528,2023-12-05 12:30:38
4,558.0,cycling,88.0,21.08,4.833,95,90.0,4.833,2023-12-05 12:30:39


The full timestamp value includes both date and time, which can be unwieldy for later analysis. We create a new Date column that strips out the time component, leaving just the calendar date.
This Date column then serves as the basis for a 'session' column, a simple identifier for each workout.

In [47]:
# Create a new column named 'date' and populate it from the date yyyy-mm-dd from the 'timestamp' column 
ardf_master['date'] = pd.to_datetime(ardf_master['timestamp'], errors='coerce').dt.date

For our new Session column, we take the Date value and convert it to a string. We then use a lambda d function to insert that date text after the prefix 'Session_', producing labels like "Session_2025-10-31".

In [49]:
# Create a simple session name based on date
ardf_master['session'] = ardf_master['date'].astype(str).apply(lambda d: f"Session_{d}")

In [50]:
# Reorder columns with session first
ardf_master = ardf_master[['session', 'accumulated_power', 'activity_type', 'cadence', 'distance','enhanced_speed', 'heart_rate', 'power', 'speed', 'timestamp', 'date']]

In [51]:
# Show column names and first five rows to check last few stepsardf_master
ardf_master.head()

,session,accumulated_power,activity_type,cadence,distance,enhanced_speed,heart_rate,power,speed,timestamp,date
0,Session_2023-12-05,139.0,cycling,74.0,3.58,3.583,90,139.0,3.583,2023-12-05 12:30:35,2023-12-05
1,Session_2023-12-05,268.0,cycling,78.0,7.50,3.917,91,129.0,3.917,2023-12-05 12:30:36,2023-12-05
2,Session_2023-12-05,377.0,cycling,80.0,11.72,4.222,93,109.0,4.222,2023-12-05 12:30:37,2023-12-05
3,Session_2023-12-05,468.0,cycling,84.0,16.25,4.528,93,91.0,4.528,2023-12-05 12:30:38,2023-12-05
4,Session_2023-12-05,558.0,cycling,88.0,21.08,4.833,95,90.0,4.833,2023-12-05 12:30:39,2023-12-05


In [ ]:
To complete the workflow, we export the cleaned ardf_master DataFrame to a CSV file for use in my data projects

In [52]:
ardf_master.to_csv('aerobic_recovery.csv', index=False)